# Pre-processing of hotel data (2004–2018)
The dataset was sourced from a Tableau graph in [SF City Performance Scorecard](https://sfgov.org/scorecards/tourism). 

In [1]:
# Dependencies
import pandas as pd
from dateutil import parser # to convert date strings to datetime format
import numpy as np

In [2]:
# Create a dataframe from the csv file
hotel_df = pd.read_csv("hotel_indicators.csv")
hotel_df.head()

,Date,Seasonal Adjustment,Average Daily Rate,Hotel Occupancy,Revenue per Available Hotel Room
0,07/01/2004 00:00,Seasonally Adjusted,141.6038,0.738484,104.5721
1,08/01/2004 00:00,Seasonally Adjusted,143.1187,0.719571,102.9840
2,09/01/2004 00:00,Seasonally Adjusted,137.4212,0.750757,103.1699
3,10/01/2004 00:00,Seasonally Adjusted,141.5987,0.778466,110.2297
4,11/01/2004 00:00,Seasonally Adjusted,148.4632,0.682347,101.3035


### Cleaning the dataframe
- Removing irrelevant variables
- Convert dates to a yyyy-bb format

In [3]:
# Choose only Date, Average Daily Rate, and Hotel Occupancy
hotel_df2 = hotel_df[["Date", "Average Daily Rate", "Hotel Occupancy"]]

In [4]:
# Format the Date to Year-Month
periods = hotel_df2["Date"].str.replace("/","-")
periods = [date[0:10] for date in periods] # excludes the time stamps 

In [5]:
# Convert the MM/DD/YYYY format to YYYY-MM
periods = [parser.parse(date).strftime('%Y-%m') for date in periods]

In [6]:
# Insert the new date format into the dataframe and drop the original Date column
hotel_df2 = hotel_df2.drop(["Date"], axis = 1)
hotel_df2["Date"] = periods
hotel_df2.head()

,Average Daily Rate,Hotel Occupancy,Date
0,141.6038,0.738484,2004-07
1,143.1187,0.719571,2004-08
2,137.4212,0.750757,2004-09
3,141.5987,0.778466,2004-10
4,148.4632,0.682347,2004-11


### Converting average daily rates to monthly rates
- Create subsets for months that don't change number of days
- Create a subset for February (which has different days depending on the year)

#### Regular months (30 and 31 days)

In [7]:
# Create a list of months in numerical format
month_30 = ["04","06","09","11"]
month_31 = ["01","03","05","07","08","10","12"]
month_28 = ["02"]

month_by_day = [month_30, month_31]
days = [30, 31]

In [8]:
# Define a function that extracts the month (2-digit format) from the Date column
def months(df,list):
    x = [df.loc[df["Date"].str.slice(-2) == item] for item in list]
    return x

In [9]:
# Create lists for months with 30 and 31 days whatever the year
month_grps = [months(hotel_df2,grp) for grp in month_by_day] # groups of months based on number of days

avg_daily = [[(month["Average Daily Rate"]) for month in grp] for grp in month_grps] # daily average rates in each group
mo_rate_30 = [(i * days[0]) for i in avg_daily[0]] # monthly rates in the 30-day group       
mo_rate_31 = [(i * days[1]) for i in avg_daily[1]] # monthly rates in the 31-day group

In [10]:
# Create dataframes for months grouped by number of days (30,31)
df30 = pd.concat(month_grps[0]) # puts all values in the same columns
df30["Monthly Rate"] = pd.concat(mo_rate_30) # add the monthly rate as an additional column

df31 = pd.concat(month_grps[1]) # puts all values in the same columns
df31["Monthly Rate"] = pd.concat(mo_rate_31) # add the monthly rate as an additional column

df31.head()

,Average Daily Rate,Hotel Occupancy,Date,Monthly Rate
6,148.8121,0.729100,2005-01,4613.1751
18,160.7223,0.755878,2006-01,4982.3913
30,179.4341,0.727883,2007-01,5562.4571
42,184.5935,0.785091,2008-01,5722.3985
54,169.4801,0.742489,2009-01,5253.8831


#### For February, if the year falls on a leap year, the number of days is 29. If not, 28.

In [11]:
# Extract a dataframe that contains the data for Feb
Feb = months(hotel_df2,month_28)
Feb = Feb[0]

,Average Daily Rate,Hotel Occupancy,Date
7,151.1379,0.740805,2005-02
19,161.3353,0.723245,2006-02
31,186.9618,0.791289,2007-02
43,197.1489,0.811044,2008-02
55,157.9055,0.648616,2009-02
67,150.5339,0.762755,2010-02
79,181.8734,0.837384,2011-02
91,213.3869,0.822019,2012-02
103,201.8483,0.854944,2013-02
115,234.2935,0.898843,2014-02


In [12]:
# Define a function that creates a list of leap years 
# Resource: https://stackoverflow.com/a/18951699/10205963
def leap(start,end):
    return [year for year in range(start, end + 1)
            if (year % 400 == 0) or (year % 4 == 0 and not year % 100 == 0)]

In [15]:
# Create a list of years from the Date column for Feb
years = [date[0:4] for date in Feb["Date"]] 
    # Feb[0] is a dataframe inside the Feb list
    # date[0:4] gives substrings from the Date column
    
years = [int(i) for i in years] # convert list of strings to list of int

In [16]:
# Get the leap years from the list of years
leap_yrs = leap(years[0], years[-1]) # years[0] = 1st; years[-1] = last
leap_yrs = [str(i) for i in leap_yrs]

In [17]:
# Create a subset for Feb data for leap years
leap_Feb = Feb.loc[Feb["Date"].str.contains("|".join(leap_yrs))]

# Calculate monthly rates for Feb leap years
leap_Feb_mo_rate = [(i * 29) for i in leap_Feb["Average Daily Rate"]] 

# Add the Monthly Rates as a new column
# Resource: https://stackoverflow.com/a/47694287/10205963

se = pd.Series(leap_Feb_mo_rate) # Convert the monthly rate list to a series
leap_Feb.insert(loc = 3, column = "Monthly Rate", value = se.values) # add the column Monthly Rate (no errors)
    
leap_Feb

,Average Daily Rate,Hotel Occupancy,Date,Monthly Rate
43,197.1489,0.811044,2008-02,5717.3181
91,213.3869,0.822019,2012-02,6188.2201
139,315.3700,0.935458,2016-02,9145.7300


In [20]:
# Create a subset for Feb data for regular years
reg_Feb = Feb.loc[~Feb["Date"].str.contains("|".join(leap_yrs))]

# Calculate monthly rates for Feb regular years
reg_Feb_mo_rate = [(i * 28) for i in reg_Feb["Average Daily Rate"]] 

# Add the Monthly Rates as a new column

se2 = pd.Series(reg_Feb_mo_rate) # Convert the monthly rate list to a series
reg_Feb.insert(loc = 3, column = "Monthly Rate", value = se2.values) # add the column Monthly Rate (no errors)
    
reg_Feb

,Average Daily Rate,Hotel Occupancy,Date,Monthly Rate
7,151.1379,0.740805,2005-02,4231.8612
19,161.3353,0.723245,2006-02,4517.3884
31,186.9618,0.791289,2007-02,5234.9304
55,157.9055,0.648616,2009-02,4421.3540
67,150.5339,0.762755,2010-02,4214.9492
79,181.8734,0.837384,2011-02,5092.4552
103,201.8483,0.854944,2013-02,5651.7524
115,234.2935,0.898843,2014-02,6560.2180
127,236.7917,0.883478,2015-02,6630.1676
151,292.0611,0.914510,2017-02,8177.7108


In [24]:
# Concatenate the two dataframes together (Feb)
dfFeb = pd.concat([leap_Feb,reg_Feb])
dfFeb

,Average Daily Rate,Hotel Occupancy,Date,Monthly Rate
43,197.1489,0.811044,2008-02,5717.3181
91,213.3869,0.822019,2012-02,6188.2201
139,315.3700,0.935458,2016-02,9145.7300
7,151.1379,0.740805,2005-02,4231.8612
19,161.3353,0.723245,2006-02,4517.3884
31,186.9618,0.791289,2007-02,5234.9304
55,157.9055,0.648616,2009-02,4421.3540
67,150.5339,0.762755,2010-02,4214.9492
79,181.8734,0.837384,2011-02,5092.4552
103,201.8483,0.854944,2013-02,5651.7524


In [26]:
# Concatenate all dataframes (30-day, 31-day, Feb)
hotel_df3 = pd.concat([df30, df31, dfFeb])
hotel_df3.head()

,Average Daily Rate,Hotel Occupancy,Date,Monthly Rate
9,157.8543,0.740657,2005-04,4735.6290
21,169.9458,0.770442,2006-04,5098.3740
33,183.4274,0.764485,2007-04,5502.8220
45,183.7442,0.768456,2008-04,5512.3260
57,157.9462,0.743636,2009-04,4738.3860
69,151.1186,0.789306,2010-04,4533.5580
81,166.3684,0.800227,2011-04,4991.0520
93,187.7816,0.796256,2012-04,5633.4480
105,225.7937,0.889583,2013-04,6773.8110
117,237.8444,0.880647,2014-04,7135.3320


In [ ]:
# Save the file in the Data folder, ready for analyses and visualisation
#hotel_df3.to_csv("../Data/hotel_rates.csv", sep = ",", encoding = "utf-8")